**Import statements**


In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

import constants
import functions3 as functions

%matplotlib inline

**Load Data**

In [2]:
all_static_data = functions.load_static_sensors_calibrated()
all_static_data = functions.transform_to_grid_coordinates(all_static_data)
mobile_sensor_data = functions.load_mobile_sensors()
mobile_sensor_data = functions.transform_to_grid_coordinates(mobile_sensor_data)

**Compute predictions for stationary sensors**

In [3]:
# Hyperparameter search definition

cs=[0.0000002, 0.00002, 0.0002, 0.002, 0.02, 0.2, 2]
epsilons=[0.001,0.1, 0.5, 1, 2.5, 5]
# This is not needed, it's been verified it works better with epsilon_insensitive
# Also saves time
#losses=['epsilon_insensitive','squared_epsilon_insensitive']
losses=['epsilon_insensitive']
include_temp_hum = [False, True]
input_window_sizes = range(8,50, 8)

In [4]:
def test_on_data(c, epsilon, loss, temp_hum, window_size, all_static_data):
    mae = []
    mse = []
    squares = []
    pred=[]

    start_time = '2018-07-03 01:00:00'
    start_window = '2018-07-05 12:00:00'
    end_window = '2018-07-05 12:15:00'
    offset_timeint = 1

    number_of_windows = 100
    window = 15
    par_grid = functions.create_par(c, epsilon, loss)
    timeint_on_first_window = 0

    for i in range(1, number_of_windows+1):
        end_time = start_window
        # treinar sem dados moveis
        # testar em dados fixos em vez de móveis
        # Aqui rolling offset para definir a rolling mean
        timeint, par_grid = functions.train(all_static_data, None, start_time, end_time, par_grid, window, timeint_on_first_window, rolling_offset=45)
        # Usar test window start e end para fazer a previsão para daqui a 1 hora e actualizar o offset na test_mobile function call.
        #test_window_start = (datetime.strptime(start_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=45)).strftime("%Y-%m-%d %H:%M:%S")
        #test_window_end = (datetime.strptime(end_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=45)).strftime("%Y-%m-%d %H:%M:%S")
        to_test = functions.test_mobile(par_grid, timeint, all_static_data, start_window, end_window, 1)
        squares.append(to_test.shape[0])
        mae.append(mean_absolute_error(to_test['PM2.5'], to_test['pred_PM2.5']))
        mse.append(mean_squared_error(to_test['PM2.5'], to_test['pred_PM2.5']))
        pred.append(to_test['pred_PM2.5'].values[0])

        start_time = end_time
        start_window = (datetime.strptime(start_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=15)).strftime("%Y-%m-%d %H:%M:%S")
        end_window = (datetime.strptime(end_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=15)).strftime("%Y-%m-%d %H:%M:%S")
        timeint_on_first_window = timeint+1
    return np.mean(mae), np.mean(mse)

In [5]:
results = pd.DataFrame(columns=["C", "epsilon", "loss", "temp_hum", "window_size", "mae", "mse"])
for c in cs:
    for epsilon in epsilons:
        for loss in losses:
            # for temp_hum in include_temp_hum:
                # for window_size in input_window_sizes:
                    temp_hum=False
                    window_size=0
                    mae, mse = test_on_data(c, epsilon, loss, temp_hum, window_size, all_static_data)
                    results = results.append({'C': c,
                                             'epsilon': epsilon,
                                             'loss': loss,
                                             'temp_hum': temp_hum,
                                             'window_size': window_size,
                                             'mae': mae,
                                             'mse': mse}, ignore_index=True)
                    print(mae)

0.3141736488149044
0.31678449077943216
0.45521936627821785
0.6153825382174954
1.023627802890817
1.023627802890817
0.12123194335261368
0.17185796071975804
0.38243481630724374
0.5849517104059876
1.023627802890817
1.023627802890817
0.12123155011254193
0.17185775614306295
0.3824530596838702
0.5847724691209158
1.023627802890817
1.023627802890817
0.12122968072179008
0.1718567867364541
0.382537325911166
0.5839569433473253
1.023627802890817
1.023627802890817
0.12122197086967085
0.17184851122560418
0.38284049899559747
0.5812081618062526
1.023627802890817
1.023627802890817
0.1211978428859848
0.1717868228277466
0.38364398718890336
0.5718544404584945
1.023627802890817
1.023627802890817
0.121174143513995
0.1716229388431487
0.3848240460968904
0.5718544404584945
1.023627802890817
1.023627802890817


In [6]:
results.to_csv('results/time_train1h.csv', index=False)
results

,C,epsilon,loss,temp_hum,window_size,mae,mse
0,2.000000e-07,0.001,epsilon_insensitive,False,0,0.314174,0.126361
1,2.000000e-07,0.100,epsilon_insensitive,False,0,0.316784,0.131170
2,2.000000e-07,0.500,epsilon_insensitive,False,0,0.455219,0.285609
3,2.000000e-07,1.000,epsilon_insensitive,False,0,0.615383,0.598538
4,2.000000e-07,2.500,epsilon_insensitive,False,0,1.023628,1.211358
5,2.000000e-07,5.000,epsilon_insensitive,False,0,1.023628,1.211358
6,2.000000e-05,0.001,epsilon_insensitive,False,0,0.121232,0.025023
7,2.000000e-05,0.100,epsilon_insensitive,False,0,0.171858,0.040615
8,2.000000e-05,0.500,epsilon_insensitive,False,0,0.382435,0.181852
9,2.000000e-05,1.000,epsilon_insensitive,False,0,0.584952,0.476694
